Import Libraries and Scrape Table Cells 

In [ ]:
import lxml.html as lh
import requests
import pandas as pd 

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

Parse Table Header 

In [111]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Creating Pandas Dataframe 

In [55]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

Rename column headers and drop not assigned from rows 

In [118]:
df.rename(columns ={'Neighbourhood':'Neighborhood','Postcode':'PostalCode'
                          }, 
                 inplace=True)
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

Group by Postcode and Borough to aggregate Neighborhoods

In [122]:
df_1 = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()

In [123]:
df_1.shape

(103, 3)